In [ ]:
# Step 1: Install required libraries
!pip install duckdb pandas --quiet

import duckdb
import pandas as pd
from google.colab import files

# Step 2: Upload CSV files from local drive
uploaded_files = files.upload()

# Step 3: Load CSV files into Pandas DataFrames
brands = pd.read_csv('brands.csv')
categories = pd.read_csv('categories.csv')
customers = pd.read_csv('customers.csv')
order_items = pd.read_csv('order_items.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')
staffs = pd.read_csv('staffs.csv')
stocks = pd.read_csv('stocks.csv')
stores = pd.read_csv('stores.csv')

# Step 4: Create DuckDB connection in-memory
con = duckdb.connect(database=':memory:')

# Step 5: Create DIMENSION tables (NO foreign keys between dimensions)

# Brands Dimension
con.execute("""
CREATE TABLE brands (
    brand_id INTEGER PRIMARY KEY,
    brand_name VARCHAR
);
""")

# Categories Dimension
con.execute("""
CREATE TABLE categories (
    category_id INTEGER PRIMARY KEY,
    category_name VARCHAR
);
""")

# Customers Dimension
con.execute("""
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    first_name VARCHAR,
    last_name VARCHAR,
    phone VARCHAR,
    email VARCHAR,
    street VARCHAR,
    city VARCHAR,
    state VARCHAR,
    zip_code VARCHAR
);
""")

# Orders Dimension
con.execute("""
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_status INTEGER,
    order_date DATE,
    required_date DATE,
    shipped_date DATE,
    store_id INTEGER,
    staff_id INTEGER
);
""")

# Products Dimension
con.execute("""
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name VARCHAR,
    brand_id INTEGER,
    category_id INTEGER,
    model_year INTEGER,
    list_price DOUBLE
);
""")

# Staffs Dimension
con.execute("""
CREATE TABLE staffs (
    staff_id INTEGER PRIMARY KEY,
    first_name VARCHAR,
    last_name VARCHAR,
    email VARCHAR,
    phone VARCHAR,
    active INTEGER,
    store_id INTEGER,
    manager_id INTEGER
);
""")

# Stores Dimension
con.execute("""
CREATE TABLE stores (
    store_id INTEGER PRIMARY KEY,
    store_name VARCHAR,
    phone VARCHAR,
    email VARCHAR,
    street VARCHAR,
    city VARCHAR,
    state VARCHAR,
    zip_code VARCHAR
);
""")

# Stocks Dimension
con.execute("""
CREATE TABLE stocks (
    stock_id INTEGER PRIMARY KEY,
    store_id INTEGER,
    product_id INTEGER,
    quantity INTEGER
);
""")

# Step 6: Create FACT table with ONLY foreign keys + measures
con.execute("""
CREATE TABLE order_items_fact (
    order_id INTEGER,
    item_id INTEGER,
    product_id INTEGER,
    brand_id INTEGER,
    category_id INTEGER,
    customer_id INTEGER,
    staff_id INTEGER,
    store_id INTEGER,
    stock_id INTEGER,
    quantity INTEGER,
    list_price DOUBLE,
    discount DOUBLE,
    PRIMARY KEY(order_id, item_id),
    FOREIGN KEY(order_id) REFERENCES orders(order_id),
    FOREIGN KEY(product_id) REFERENCES products(product_id),
    FOREIGN KEY(brand_id) REFERENCES brands(brand_id),
    FOREIGN KEY(category_id) REFERENCES categories(category_id),
    FOREIGN KEY(customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY(staff_id) REFERENCES staffs(staff_id),
    FOREIGN KEY(store_id) REFERENCES stores(store_id),
    FOREIGN KEY(stock_id) REFERENCES stocks(stock_id)
);
""")

# Step 7: Insert data into dimension tables first
con.register('brands_df', brands)
con.execute('INSERT INTO brands SELECT * FROM brands_df')

con.register('categories_df', categories)
con.execute('INSERT INTO categories SELECT * FROM categories_df')

con.register('customers_df', customers)
con.execute('INSERT INTO customers SELECT * FROM customers_df')

con.register('orders_df', orders)
con.execute('INSERT INTO orders SELECT * FROM orders_df')

con.register('products_df', products)
con.execute('INSERT INTO products SELECT * FROM products_df')

con.register('staffs_df', staffs)
con.execute('INSERT INTO staffs SELECT * FROM staffs_df')

con.register('stores_df', stores)
con.execute('INSERT INTO stores SELECT * FROM stores_df')

# Add stock_id to stocks if missing and insert
if 'stock_id' not in stocks.columns:
    stocks = stocks.reset_index().rename(columns={'index': 'stock_id'})
con.register('stocks_df', stocks)
con.execute('INSERT INTO stocks SELECT * FROM stocks_df')

# Step 8: Create enriched fact table data by merging foreign keys
# Merge order_items with orders to get customer_id, staff_id, store_id
fact_data = order_items.merge(
    orders[['order_id', 'customer_id', 'staff_id', 'store_id']],
    on='order_id',
    how='left'
)

# Merge with products to get brand_id, category_id
fact_data = fact_data.merge(
    products[['product_id', 'brand_id', 'category_id']],
    on='product_id',
    how='left'
)

# Merge with stocks to get stock_id (assuming we need this relationship)
fact_data = fact_data.merge(
    stocks[['store_id', 'product_id', 'stock_id']],
    on=['store_id', 'product_id'],
    how='left'
)

# Fill missing stock_id with a default value if no match
fact_data['stock_id'] = fact_data['stock_id'].fillna(0).astype(int)

# Step 9: Insert into fact table
con.register('fact_data_df', fact_data)
con.execute("""
INSERT INTO order_items_fact
(order_id, item_id, product_id, brand_id, category_id, customer_id, staff_id, store_id, stock_id, quantity, list_price, discount)
SELECT
    order_id, item_id, product_id, brand_id, category_id, customer_id, staff_id, store_id, stock_id, quantity, list_price, discount
FROM fact_data_df
""")

# Step 10: Verify star schema structure
print("DIMENSION TABLES:")
print("Brands:", con.execute("SELECT COUNT(*) FROM brands").fetchone()[0])
print("Categories:", con.execute("SELECT COUNT(*) FROM categories").fetchone()[0])
print("Customers:", con.execute("SELECT COUNT(*) FROM customers").fetchone()[0])
print("Orders:", con.execute("SELECT COUNT(*) FROM orders").fetchone()[0])
print("Products:", con.execute("SELECT COUNT(*) FROM products").fetchone()[0])
print("Staffs:", con.execute("SELECT COUNT(*) FROM staffs").fetchone()[0])
print("Stores:", con.execute("SELECT COUNT(*) FROM stores").fetchone()[0])
print("Stocks:", con.execute("SELECT COUNT(*) FROM stocks").fetchone()[0])
print("\nFACT TABLE:")
print("Order Items Fact:", con.execute("SELECT COUNT(*) FROM order_items_fact").fetchone()[0])




Saving brands.csv to brands (1).csv
Saving categories.csv to categories (1).csv
Saving customers.csv to customers (1).csv
Saving order_items.csv to order_items (1).csv
Saving orders.csv to orders (1).csv
Saving products.csv to products (1).csv
Saving staffs.csv to staffs (1).csv
Saving stocks.csv to stocks (1).csv
Saving stores.csv to stores (1).csv
Total Sales Revenue per Year, Quarter, Month:
     year  quarter  month  total_revenue
0   2016        1      1    215146.4241
1   2016        1      2    156112.3228
2   2016        1      3    180600.3285
3   2016        1   <NA>    551859.0754
4   2016        2      4    167144.0512
5   2016        2      5    205270.0091
6   2016        2      6    210562.1245
7   2016        2   <NA>    582976.1848
8   2016        3      7    199556.8089
9   2016        3      8    225657.3767
10  2016        3      9    273091.6097
11  2016        3   <NA>    698305.7953
12  2016        4     10    212078.0805
13  2016        4     11    182329.4124
1

In [ ]:
query = """
SELECT
    EXTRACT(year FROM order_date) AS year,
    EXTRACT(quarter FROM order_date) AS quarter,
    EXTRACT(month FROM order_date) AS month,
    SUM(quantity * list_price * (1 - discount)) AS total_revenue
FROM order_items
GROUP BY ROLLUP(year, quarter, month)
ORDER BY year, quarter, month;
"""
result = con.execute(query).df()
print(result)

    year  quarter  month  total_revenue
0   2016        1      1   2.151464e+05
1   2016        1      2   1.561123e+05
2   2016        1      3   1.806003e+05
3   2016        1   <NA>   5.518591e+05
4   2016        2      4   1.671441e+05
5   2016        2      5   2.052700e+05
6   2016        2      6   2.105621e+05
7   2016        2   <NA>   5.829762e+05
8   2016        3      7   1.995568e+05
9   2016        3      8   2.256574e+05
10  2016        3      9   2.730916e+05
11  2016        3   <NA>   6.983058e+05
12  2016        4     10   2.120781e+05
13  2016        4     11   1.823294e+05
14  2016        4     12   1.998300e+05
15  2016        4   <NA>   5.942375e+05
16  2016     <NA>   <NA>   2.427379e+06
17  2017        1      1   2.856165e+05
18  2017        1      2   3.129237e+05
19  2017        1      3   3.089119e+05
20  2017        1   <NA>   9.074521e+05
21  2017        2      4   2.272909e+05
22  2017        2      5   2.682332e+05
23  2017        2      6   3.788657e+05


In [ ]:
query = """
WITH monthly_sales AS (
    SELECT
        EXTRACT(year FROM order_date) AS year,
        EXTRACT(month FROM order_date) AS month,
        SUM(quantity * list_price * (1 - discount)) AS total_revenue
    FROM order_items
    GROUP BY year, month
)
SELECT year, month, total_revenue
FROM (
    SELECT
        year,
        month,
        total_revenue,
        RANK() OVER (PARTITION BY year ORDER BY total_revenue DESC) AS rank
    FROM monthly_sales
) AS ranked_sales
WHERE rank = 1
ORDER BY year;
"""

result = con.execute(query).df()
print(result)

   year  month  total_revenue
0  2016      9    273091.6097
1  2017      6    378865.6535
2  2018      4    817921.8604


In [ ]:
import textwrap

query = textwrap.dedent("""
    SELECT
        c.category_name,
        SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS revenue
    FROM order_items oi
    JOIN categories c ON oi.category_id = c.category_id
    GROUP BY c.category_name
    ORDER BY revenue DESC;
""")

result = con.execute(query).df()
print(result)

         category_name       revenue
0       Mountain Bikes  2.715080e+06
1           Road Bikes  1.665098e+06
2    Cruisers Bicycles  9.950326e+05
3       Electric Bikes  9.166848e+05
4  Cyclocross Bicycles  7.110118e+05
5     Comfort Bicycles  3.940201e+05
6    Children Bicycles  2.921892e+05


In [ ]:
import textwrap

query = textwrap.dedent("""
    SELECT
        c.category_name,
        SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS revenue
    FROM order_items oi
    JOIN categories c ON oi.category_id = c.category_id
    GROUP BY c.category_name
    ORDER BY revenue DESC;
""")

result = con.execute(query).df()
print(result)

         category_name       revenue
0       Mountain Bikes  2.715080e+06
1           Road Bikes  1.665098e+06
2    Cruisers Bicycles  9.950326e+05
3       Electric Bikes  9.166848e+05
4  Cyclocross Bicycles  7.110118e+05
5     Comfort Bicycles  3.940201e+05
6    Children Bicycles  2.921892e+05


In [ ]:
import textwrap

query = textwrap.dedent("""
    SELECT
        c.category_name,
        p.product_name,
        SUM(oi.quantity) AS total_quantity_sold,
        SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS revenue
    FROM order_items oi
    JOIN categories c ON oi.category_id = c.category_id
    JOIN products p ON oi.product_id = p.product_id
    GROUP BY c.category_name, p.product_name
    ORDER BY c.category_name, revenue DESC;
""")

result = con.execute(query).df()
print(result)

         category_name                                   product_name  \
0    Children Bicycles  Electra Girl's Hawaii 1 (20-inch) - 2015/2016   
1    Children Bicycles  Electra Girl's Hawaii 1 (16-inch) - 2015/2016   
2    Children Bicycles             Electra Cruiser 1 (24-Inch) - 2016   
3    Children Bicycles  Electra Townie 3i EQ (20-inch) - Boys' - 2017   
4    Children Bicycles             Electra Girl's Hawaii 1 16" - 2017   
..                 ...                                            ...   
302         Road Bikes           Trek Domane SLR Disc Frameset - 2018   
303         Road Bikes                Trek Domane SL 5 Women's - 2018   
304         Road Bikes                         Trek CrossRip 2 - 2018   
305         Road Bikes                       Trek Domane ALR 3 - 2018   
306         Road Bikes                      Surly ECR Frameset - 2018   

     total_quantity_sold     revenue  
0                  154.0  41011.6329  
1                  145.0  34728.8137  
2     

In [ ]:
import textwrap

query = textwrap.dedent("""
    SELECT
        b.brand_name,
        c.category_name,
        EXTRACT(year FROM oi.order_date) AS year,
        SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS total_revenue
    FROM order_items oi
    JOIN brands b ON oi.brand_id = b.brand_id
    JOIN categories c ON oi.category_id = c.category_id
    GROUP BY CUBE (b.brand_name, c.category_name, year)
    ORDER BY b.brand_name NULLS LAST, c.category_name NULLS LAST, year NULLS LAST;
""")

result = con.execute(query).df()
print(result)

    brand_name      category_name  year  total_revenue
0      Electra  Children Bicycles  2016   8.768087e+04
1      Electra  Children Bicycles  2017   8.679506e+04
2      Electra  Children Bicycles  2018   3.312979e+04
3      Electra  Children Bicycles  <NA>   2.076057e+05
4      Electra   Comfort Bicycles  2016   1.544872e+05
..         ...                ...   ...            ...
135       None         Road Bikes  <NA>   1.665098e+06
136       None               None  2016   2.427379e+06
137       None               None  2017   3.447208e+06
138       None               None  2018   1.814530e+06
139       None               None  <NA>   7.689117e+06

[140 rows x 4 columns]


In [ ]:
import textwrap

query = textwrap.dedent("""
    SELECT
        cu.customer_id,
        cu.first_name,
        cu.last_name,
        SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS total_purchase
    FROM order_items oi
    JOIN customers cu ON oi.customer_id = cu.customer_id
    GROUP BY cu.customer_id, cu.first_name, cu.last_name
    ORDER BY total_purchase DESC
    LIMIT 5;
""")

result = con.execute(query).df()
print(result)

   customer_id first_name last_name  total_purchase
0           94     Sharyn   Hopkins      34807.9392
1           10    Pamelia    Newman      33634.2604
2           75       Abby    Gamble      32803.0062
3            6    Lyndsey      Bean      32675.0725
4           16     Emmitt   Sanchez      31925.8857


In [ ]:
import textwrap

query = textwrap.dedent("""
    SELECT
        s.staff_id,
        s.first_name,
        s.last_name,
        st.store_name,
        SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS total_revenue
    FROM order_items oi
    JOIN staffs s ON oi.staff_id = s.staff_id
    JOIN stores st ON oi.store_id = st.store_id
    GROUP BY s.staff_id, s.first_name, s.last_name, st.store_name
    ORDER BY st.store_name, total_revenue DESC;
""")

result = con.execute(query).df()
print(result)

   staff_id first_name last_name        store_name  total_revenue
0         6  Marcelene     Boyer     Baldwin Bikes   2.624121e+06
1         7     Venita    Daniel     Baldwin Bikes   2.591631e+06
2         8       Kali    Vargas     Rowlett Bikes   4.639183e+05
3         9      Layla   Terrell     Rowlett Bikes   4.036239e+05
4         3      Genna   Serrano  Santa Cruz Bikes   8.532874e+05
5         2     Mireya  Copeland  Santa Cruz Bikes   7.525357e+05


In [ ]:
import textwrap

query = textwrap.dedent("""
    SELECT
        c.category_name,
        st.store_name,
        EXTRACT(year FROM oi.order_date) AS year,
        SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS total_revenue
    FROM order_items oi
    JOIN categories c ON oi.category_id = c.category_id
    JOIN stores st ON oi.store_id = st.store_id
    GROUP BY CUBE(c.category_name, st.store_name, year)
    ORDER BY c.category_name NULLS LAST, st.store_name NULLS LAST, year NULLS LAST;
""")

result = con.execute(query).df()
print(result)

         category_name        store_name  year  total_revenue
0    Children Bicycles     Baldwin Bikes  2016   6.218418e+04
1    Children Bicycles     Baldwin Bikes  2017   1.034127e+05
2    Children Bicycles     Baldwin Bikes  2018   3.705504e+04
3    Children Bicycles     Baldwin Bikes  <NA>   2.026519e+05
4    Children Bicycles     Rowlett Bikes  2016   8.321401e+03
..                 ...               ...   ...            ...
119               None  Santa Cruz Bikes  <NA>   1.605823e+06
120               None              None  2016   2.427379e+06
121               None              None  2017   3.447208e+06
122               None              None  2018   1.814530e+06
123               None              None  <NA>   7.689117e+06

[124 rows x 4 columns]


In [ ]:
import textwrap

query = textwrap.dedent("""
    SELECT
        st.store_name,
        SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS total_revenue
    FROM order_items oi
    JOIN stores st ON oi.store_id = st.store_id
    GROUP BY CUBE(st.store_name)
    ORDER BY st.store_name NULLS LAST;
""")

result = con.execute(query).df()
print(result)

         store_name  total_revenue
0     Baldwin Bikes   5.215751e+06
1     Rowlett Bikes   8.675422e+05
2  Santa Cruz Bikes   1.605823e+06
3              None   7.689117e+06


In [ ]:
import textwrap

query = textwrap.dedent("""
    SELECT
        EXTRACT(year FROM oi.order_date) AS year,
        SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS total_revenue
    FROM order_items oi
    GROUP BY CUBE(year)
    ORDER BY year NULLS LAST;
""")

result = con.execute(query).df()
print(result)

   year  total_revenue
0  2016   2.427379e+06
1  2017   3.447208e+06
2  2018   1.814530e+06
3  <NA>   7.689117e+06
